In [ ]:
from flask import Flask, request, render_template
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

app = Flask(__name__)

# Load models and ensemble weights
trained_xgb = joblib.load("trained_xgb_model.pkl")


# Load the pre-fitted scaler
scaler = joblib.load("scaler.pkl")

column_order = joblib.load("column_order.pkl")

@app.route('/')
def index():
    return render_template('testnow.html')

@app.route('/process', methods=['POST'])
def process():
    try:
        # Collect form data
        Age = int(request.form.get('age', 0))
        Gender = request.form.get('gender', '').upper()
        MaritalStatus = request.form.get('marital-status', '').upper()
        Ethnicity = request.form.get('ethnicity', '').upper()
        Clarity = request.form.get('clarity', '').upper()
        M_Color = request.form.get('color', '').upper()
        Transparency = request.form.get('transparency', '').upper()
        SpecificGravity = float(request.form.get('specific-gravity', 1.0))
        PHLevel = float(request.form.get('ph-level', 7.0))
        Leukocytes = request.form.get('leukocytes', '').upper()
        Nitrite = request.form.get('nitrite', '').upper()
        Protein = request.form.get('protein', '').upper()
        Glucose = request.form.get('glucose', '').upper()
        Ketones = request.form.get('ketones', '').upper()
        Urobilinogen = request.form.get('urobilinogen', '').upper()
        Bilirubin = request.form.get('bilirubin', '').upper()
        Blood = request.form.get('blood', '').upper()
        Bacteria = request.form.get('bacteria', '').upper()
        EpithilialCells = request.form.get('epithelial-cells', '').upper()
        MucusThreads = request.form.get('mucus-threads', '').upper()
        AmorphousUrate = request.form.get('amorphous-urate', '').upper()
        RedBC = request.form.get('rbc', '').upper()
        WhiteBC = request.form.get('wbc', '').upper()

        # Prepare data for prediction
        sample_data = {
            'Age': [Age],
            'Sex': [Gender],
            'Ethnicity': [Ethnicity],
            'Marital Status': [MaritalStatus],
            'Clarity': [Clarity],
            'Color': [M_Color],
            'Transparency': [Transparency],
            'Spec Gravity': [SpecificGravity],
            'pH Level': [PHLevel],
            'Leukocytes': [Leukocytes],
            'Nitrite': [Nitrite],
            'Protein': [Protein],
            'Glucose': [Glucose],
            'Ketones': [Ketones],
            'Urobilinogen': [Urobilinogen],
            'Bilirubin': [Bilirubin],
            'Blood': [Blood],
            'Bacteria': [Bacteria],
            'Epithelial Cells': [EpithilialCells],
            'RBC': [RedBC],
            'WBC': [WhiteBC],
            'Mucus Threads': [MucusThreads],
            'Amorphous Urate/Phosphate': [AmorphousUrate]
        }
        sample_df = pd.DataFrame(sample_data)

        # Preprocessing the data
        sample_df = preprocess_data(sample_df)

        # Scale the data
        X_test = scaler.transform(sample_df)

        # Make predictions
        predictions = ensemble_predict(X_test)

        # print(f"Predicted Class: {predictions}")
        # return f"Prediction: {predictions}"
    
        response = f"Predicted Class: {predictions}"
        return render_template('testnow.html', response=predictions)

    except Exception as e:
        print(f"Error: {e}")
        return "An error occurred during processing."

def preprocess_data(df):
    # Convert binary categorical columns
    binary_columns = {'Clarity': {'INCREASED': 1, 'DECREASED': 0, 'CLEAR': 0}}
    for col, mapping in binary_columns.items():
        df[col] = df[col].map(mapping)

    # Handle 'Urobilinogen' column specifically
    df['Urobilinogen'] = df['Urobilinogen'].replace('NORMAL', 1)
    # Convert any other non-numeric values to 0 (or adjust as needed)
    df['Urobilinogen'] = pd.to_numeric(df['Urobilinogen'], errors='coerce').fillna(0)

    # Convert ordinal categorical columns to numerical codes
    ordinal_mappings = {
        'Color': ['CLEAR', 'PALE YELLOW', 'STRAW', 'LIGHT YELLOW', 'YELLOW', 'DARK YELLOW', 'AMBER', 'BROWN', 'RED'],
        'Transparency': ['CLEAR', 'HAZY', 'CLOUDY', 'MILKY', 'TURBID', 'RED'],
        'Nitrite': ['NEGATIVE', 'TRACE', 'POSITIVE'],
        'Protein': ['NEGATIVE', 'TRACE', '1+', '2+', '3+', '4+'],
        'Leukocytes': ['NEGATIVE', 'TRACE', '1+', '2+', '3+', '4+'],
        'Glucose': ['NEGATIVE', 'TRACE', '1+', '2+', '3+', '4+'],
        'Ketones': ['NEGATIVE', 'TRACE', '1+', '2+', '3+', '4+'],
        'Bilirubin': ['NEGATIVE', 'TRACE', '1+', '2+', '3+', '4+'],
        'Blood': ['NEGATIVE', 'TRACE', '1+', '2+', '3+', '4+'],
        'Bacteria': ['RARE', 'FEW', 'MODERATE', 'MANY', 'TNTC'],
        'Epithelial Cells': ['RARE', 'FEW', 'MODERATE', 'MANY', 'TNTC'],
        'RBC': ['0-2', '5-10', '10-20', '20-30', 'TNTC'],
        'WBC': ['0-2', '5-10', '10-20', '20-30', 'TNTC'],
        'Mucus Threads': ['RARE', 'FEW', 'MODERATE', 'MANY', 'TNTC'],
        'Amorphous Urate/Phosphate': ['RARE', 'FEW', 'MODERATE', 'MANY', 'TNTC']
    }
    for col, order in ordinal_mappings.items():
        df[col] = pd.Categorical(df[col], categories=order, ordered=True).codes

    # Label encode binary categorical columns
    df['FEMALE'] = df['Sex'].apply(lambda x: 1 if x == 'FEMALE' else 0)
    df.drop(columns=['Sex'], inplace=True)
    
    df['FILIPINO'] = df['Ethnicity'].apply(lambda x: 1 if x == 'FILIPINO' else 0)
    df.drop(columns=['Ethnicity'], inplace=True)

    # One-hot encode the 'Marital Status' column
    df = pd.get_dummies(df, columns=['Marital Status'], drop_first=False)

    # Rename the columns if necessary
    if 'Marital Status_WIDOWED' in df.columns:
        df.rename(columns={'Marital Status_WIDOWED': 'Marital Status_WIDOW'}, inplace=True)

    # Add missing columns that were present during training
    for col in column_order:
        if col not in df.columns:
            df[col] = 0  # Add missing columns with default value 0

    # Reorder the columns to match the training set
    df = df[column_order]

    return df

def ensemble_predict(X_test):
    # Get the prediction probabilities from each model
    xgb_pred = trained_xgb.predict_proba(X_test)
  
    # Print the class probabilities for each model
    print("\nXGBoost Prediction Probabilities:")
    for i, prob in enumerate(xgb_pred[0]):
        print(f"Class {i}: {prob * 100:.2f}%")


    # Print the ensemble probabilities
    print("\nEnsemble Prediction Probabilities:")
    for i, prob in enumerate(xgb_pred[0]):
        print(f"Class {i}: {prob * 100:.2f}%")

  
    
    # Return the predicted class based on the ensemble
    return np.argmax(xgb_pred, axis=1)



# def ensemble_predict(X_test):
#     xgb_pred = trained_xgb.predict_proba(X_test)
#     lgbm_pred = trained_lgbm.predict_proba(X_test)
#     svm_pred = trained_svm.predict_proba(X_test)
#     weighted_avg = (weight_xgb * xgb_pred + weight_lgbm * lgbm_pred + weight_svc * svm_pred) / (
#         weight_xgb + weight_lgbm + weight_svc)
#     return np.argmax(weighted_avg, axis=1)

if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Nov/2024 23:36:12] "POST /process HTTP/1.1" 200 -
127.0.0.1 - - [24/Nov/2024 23:36:12] "GET /static/styles.css HTTP/1.1" 304 -



XGBoost Prediction Probabilities:
Class 0: 39.75%
Class 1: 60.25%

Ensemble Prediction Probabilities:
Class 0: 39.75%
Class 1: 60.25%
